In [28]:
import numpy as np
import h5py as h5
import arepo
from numba import njit

In [5]:
basepath = '/n/holylfs05/LABS/hernquist_lab/Users/abeane/gasbar/'

snap_path = '/n/holystore01/LABS/hernquist_lab/Users/abeane/starbar_runs/runs/'
fourier_path = basepath + '/analysis/fourier_component/data/'
bprop_path = basepath + '/analysis/bar_prop/data/'
torque_path = basepath + '/analysis/torques/data/'

In [6]:
def read_torque(name, lvl):
    base = torque_path + 'torques_' + name + '-' + lvl + '/torques_' + name + '-' + lvl + '.'

    # nfiles = len(glob.glob(base + '*.hdf5'))
    nfiles = 1200

    tz_halo = []
    tz_not_bar = []
    tz_gas = []

    tlist = []

    for i in tqdm(range(nfiles)):
        fname = base + str(i) + '.hdf5'
        t = h5.File(fname, mode='r')

        torque_halo = t['total_torques'].attrs['halo']
        tz_halo.append(torque_halo[2])

        torque_not_bar = t['total_torques'].attrs['not_bar']
        tz_not_bar.append(torque_not_bar[2])
    
        if 'gas' in t['total_torques'].attrs.keys():
            torque_gas = t['total_torques'].attrs['gas']
            tz_gas.append(torque_gas[2])
        
        tlist.append(t['parameters'].attrs['Time'])
        
        t.close()
    
    tz_halo = np.array(tz_halo)
    tz_not_bar = np.array(tz_not_bar)
    tz_gas = np.array(tz_gas)
    tlist = np.array(tlist)

    return tlist, tz_halo, tz_not_bar, tz_gas

In [36]:
name = 'phantom-vacuum-Sg20-Rc3.5'
lvl = 'lvl3'

i = 301

base = torque_path + 'torques_' + name + '-' + lvl + '/torques_' + name + '-' + lvl + '.'
fname = base + str(i) + '.hdf5'
t = h5.File(fname, mode='r')

In [37]:
@njit
def my_mult(a, b):
    out = np.zeros(b.shape)
    for i in range(len(a)):
        out[i] = a[i] * b[i]
    
    return out

In [39]:
torque_gas = my_mult(t['gas_mass'][:], np.cross(t['pos_gas'], t['acc_gas']))
R = np.linalg.norm(t['pos_gas'][:,:2], axis=1)

print(np.sum(torque_gas[:,2][R > 6]))

0.8435579838433278
